# Energy bookkeeping tests for `HybridTopologyFactory`

In [1]:
import copy
from simtk import openmm, unit
from simtk.openmm import app
import os, os.path
import sys, math
import numpy as np
from openeye import oechem
from openmmtools.constants import kB
from openmmtools import alchemy, states

temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
CARBON_MASS = 12.01

## The first test is a valence-only vacuum transformation from a 3-4 bead system

The first thing to do is generating the system...

In [2]:
from perses.tests.test_geometry_engine_analytical import AnalyticalBeadSystems
ABS = AnalyticalBeadSystems([4,5], 1)
nbfA, nbfB = openmm.NonbondedForce(), openmm.NonbondedForce()


We are going to make nonbonded interactions for each and every particle in the system for bookkeeping purposes...

In [3]:
nbf = openmm.NonbondedForce()
nbf.setNonbondedMethod(0)
for _ in range(4):
    a = nbf.addParticle(1., 0.1, 1.)
    print(a)
ABS.sys_pos_top['A'][0].addForce(nbf)

0
1
2
3


3

In [4]:
nbf = openmm.NonbondedForce()
nbf.setNonbondedMethod(0)
for _ in range(5):
    a = nbf.addParticle(1., 0.1, 1.)
    print(a)
    
#add exception
nbf.addException(0,4, 4., 0.3, 2.)
ABS.sys_pos_top['B'][0].addForce(nbf)

0
1
2
3
4


3

In [5]:
ABS.sys_pos_top['A'][0].getForces()

[<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x00000232C4411570> >,
 <simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x00000232C4411210> >,
 <simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x00000232C4411F00> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x00000232C44113F0> >]

Now we can generate the topology proposal...

In [6]:
top_proposal = ABS.create_simple_topology_proposal(ABS.sys_pos_top, 4, 5)

Now we have to make the topology proposal from 3 to 4


In [7]:
from perses.rjmc.geometry import FFAllAngleGeometryEngine
_geometry_engine = FFAllAngleGeometryEngine(metadata=None, use_sterics=False, n_bond_divisions=100, n_angle_divisions=180, n_torsion_divisions=360, verbose=True, storage=None, bond_softening_constant=1.0, angle_softening_constant=1.0, neglect_angles = False)
_new_positions, _lp = _geometry_engine.propose(top_proposal, ABS.sys_pos_top['A'][1], beta)


C:\Users\domin\Anaconda3\lib\site-packages\parmed\openmm\topsystem.py:451: OpenMMWarning: Detected incomplete exceptions. Not supported.
  warnings.warn('Detected incomplete exceptions. Not supported.', OpenMMWarning)


Now that we have the proposal and the new positions, we can try to build a hybrid system


In [8]:
from perses.annihilation.new_relative import HybridTopologyFactory
HTF = HybridTopologyFactory(top_proposal, ABS.sys_pos_top['A'][1], _new_positions)

In [9]:
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import copy
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
hybrid_system = HTF.hybrid_system
lambda_zero_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_one_alchemical_state = copy.deepcopy(lambda_zero_alchemical_state)

lambda_zero_alchemical_state.set_alchemical_parameters(0.0)
lambda_one_alchemical_state.set_alchemical_parameters(1.0)

In [10]:
# Create thermodynamic states for the nonalchemical endpoints
thermodynamic_state = ThermodynamicState(hybrid_system, temperature = temperature)
nonalchemical_thermodynamic_states = {
    0: ThermodynamicState(top_proposal.old_system, temperature=temperature),
    1: ThermodynamicState(top_proposal.new_system, temperature=temperature)}

# Now create the compound states with different alchemical states
hybrid_thermodynamic_states = {0: CompoundThermodynamicState(thermodynamic_state,
                                                                   composable_states=[lambda_zero_alchemical_state]),
                                     1: CompoundThermodynamicState(copy.deepcopy(thermodynamic_state),
                                                                   composable_states=[lambda_one_alchemical_state])}


In [11]:
#create contexts
from openmmtools import integrators
from perses.tests.utils import compute_potential_components
nonalchemical_zero_context = nonalchemical_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_zero_context.setPositions(ABS.sys_pos_top['A'][1])
nonalchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_zero_context)]
print(f"nonalchemical_zero_energy components")
for force, energy in nonalchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_zero_energy: {sum(nonalchemical_zero_energy_components)}")
print()

nonalchemical_one_context = nonalchemical_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_one_context.setPositions(_new_positions)
nonalchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_one_context)]
print(f"nonalchemical_one_energy components")
for force, energy in nonalchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_one_energy: {sum(nonalchemical_one_energy_components)}")
print()
print()

alchemical_zero_context = hybrid_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_zero_context.setPositions(HTF._hybrid_positions)
alchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_zero_context)]
print(f"alchemical_zero_energy components")
for force, energy in alchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_zero_energy: {sum(alchemical_zero_energy_components)}")
print()

alchemical_one_context = hybrid_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_one_context.setPositions(HTF._hybrid_positions)
alchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_one_context)]
print(f"alchemical_one_energy components")
for force, energy in alchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_one_energy: {sum(alchemical_one_energy_components)}")

nonalchemical_zero_energy components
	HarmonicBondForce: 7.069503467623775e-06
	HarmonicAngleForce: 2.2535716234771907e-05
	PeriodicTorsionForce: 4.765657924689454e-05
	NonbondedForce: 1697.2955327570223

nonalchemical_one_energy components
	HarmonicBondForce: 0.04115371616685964
	HarmonicAngleForce: 0.33451870627409847
	PeriodicTorsionForce: 0.5474218262622164
	NonbondedForce: 3161.7937979499743


alchemical_zero_energy components
	CustomBondForce: 7.069503467623773e-06
	HarmonicBondForce: 0.041146646663392
	CustomAngleForce: 2.2535716234771907e-05
	HarmonicAngleForce: 0.3344961705578637
	CustomTorsionForce: 4.765657924689454e-05
	PeriodicTorsionForce: 0.5473741696829694
	NonbondedForce: 1697.6655976664208
	CustomNonbondedForce: -0.3700649093984062
	CustomBondForce: 0.0

alchemical_one_energy components
	CustomBondForce: 7.069503467623773e-06
	HarmonicBondForce: 0.041146646663392
	CustomAngleForce: 2.2535716234771907e-05
	HarmonicAngleForce: 0.3344961705578637
	CustomTorsionForce: 4.7

## Next test is to conduct a vacuum transformation for some small molecules and test that the $\Delta U$ between the alchemical and nonalchemical systems is just the valence terms added and subtracted

In [12]:
from perses.tests.utils import generate_vacuum_topology_proposal
from perses.rjmc.geometry import FFAllAngleGeometryEngine

In [13]:
top_proposal, old_positions, new_positions = generate_vacuum_topology_proposal('benzene', 'toluene')

DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:proposal_engine:Building new Topology object...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:proposal_engine:Proposed transformation would delete 1 atoms and create 4 atoms.


In [14]:
_geometry_engine = FFAllAngleGeometryEngine(metadata=None, use_sterics=False, n_bond_divisions=100, n_angle_divisions=180, n_torsion_divisions=360, verbose=True, storage=None, bond_softening_constant=1.0, angle_softening_constant=1.0, neglect_angles = False)

In [15]:
_new_positions, _lp = _geometry_engine.propose(top_proposal, old_positions, beta)

In [16]:
_lp_rev = _geometry_engine.logp_reverse(top_proposal, _new_positions, old_positions, beta)

In [17]:
from perses.annihilation.new_relative import HybridTopologyFactory
HTF = HybridTopologyFactory(top_proposal, old_positions, _new_positions)

In [18]:
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import copy
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from openmmtools import integrators

hybrid_system = HTF.hybrid_system
lambda_zero_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_one_alchemical_state = copy.deepcopy(lambda_zero_alchemical_state)

lambda_zero_alchemical_state.set_alchemical_parameters(0.0)
lambda_one_alchemical_state.set_alchemical_parameters(1.0)




In [19]:
# Create thermodynamic states for the nonalchemical endpoints
thermodynamic_state = ThermodynamicState(hybrid_system, temperature = temperature)
nonalchemical_thermodynamic_states = {
    0: ThermodynamicState(top_proposal.old_system, temperature=temperature),
    1: ThermodynamicState(top_proposal.new_system, temperature=temperature)}

# Now create the compound states with different alchemical states
hybrid_thermodynamic_states = {0: CompoundThermodynamicState(thermodynamic_state,
                                                                   composable_states=[lambda_zero_alchemical_state]),
                                     1: CompoundThermodynamicState(copy.deepcopy(thermodynamic_state),
                                                                   composable_states=[lambda_one_alchemical_state])}

In [20]:



#create contexts
from openmmtools import integrators
from perses.tests.utils import compute_potential_components
nonalchemical_zero_context = nonalchemical_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_zero_context.setPositions(old_positions)
nonalchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_zero_context)]
print(f"nonalchemical_zero_energy components")
for force, energy in nonalchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_zero_energy: {sum(nonalchemical_zero_energy_components)}")
print()

nonalchemical_one_context = nonalchemical_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_one_context.setPositions(_new_positions)
nonalchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_one_context)]
print(f"nonalchemical_one_energy components")
for force, energy in nonalchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_one_energy: {sum(nonalchemical_one_energy_components)}")
print()
print()

alchemical_zero_context = hybrid_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_zero_context.setPositions(HTF._hybrid_positions)
alchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_zero_context)]
print(f"alchemical_zero_energy components")
for force, energy in alchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_zero_energy: {sum(alchemical_zero_energy_components)}")
print()

alchemical_one_context = hybrid_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_one_context.setPositions(HTF._hybrid_positions)
alchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_one_context)]
print(f"alchemical_one_energy components")
for force, energy in alchemical_one_energy_components:
    print(f"\t{force}: {energy}")

nonalchemical_zero_energy components
	HarmonicBondForce: 0.05778808133185309
	HarmonicAngleForce: 0.004459776235889132
	PeriodicTorsionForce: 0.00011650644160092853
	NonbondedForce: 9.931443768311913

nonalchemical_one_energy components
	HarmonicBondForce: 0.09034441318339771
	HarmonicAngleForce: 1.6818291498411373
	PeriodicTorsionForce: 0.07718893289088935
	NonbondedForce: 4.242997802390351


alchemical_zero_energy components
	CustomBondForce: 0.05778808133185307
	HarmonicBondForce: 0.03255633185154461
	CustomAngleForce: 0.0037304346481589223
	HarmonicAngleForce: 1.6788280567807081
	CustomTorsionForce: 0.00011275108861683794
	PeriodicTorsionForce: 0.07707993715525659
	NonbondedForce: 8.514948878544267
	CustomNonbondedForce: -0.2782172927060857
	CustomBondForce: 1.6947121824737281

alchemical_one_energy components
	CustomBondForce: 0.05778808133185307
	HarmonicBondForce: 0.03255633185154461
	CustomAngleForce: 0.0037304346481589223
	HarmonicAngleForce: 1.6788280567807081
	CustomTorsionF

In [21]:
forward_added_valence_energy = _geometry_engine.forward_final_context_reduced_potential - _geometry_engine.forward_atoms_with_positions_reduced_potential
reverse_subtracted_valence_energy = _geometry_engine.reverse_final_context_reduced_potential - _geometry_engine.reverse_atoms_with_positions_reduced_potential

In [22]:
nonalch_zero_rp = SamplerState.from_context(nonalchemical_zero_context).potential_energy*beta
nonalch_one_rp = SamplerState.from_context(nonalchemical_one_context).potential_energy*beta

alch_zero_rp = SamplerState.from_context(alchemical_zero_context).potential_energy*beta
alch_one_rp = SamplerState.from_context(alchemical_one_context).potential_energy*beta

In [23]:
print(f"nonalchemical_zero: {nonalch_zero_rp}, alch_zero - forward valence: {alch_zero_rp - forward_added_valence_energy}")

nonalchemical_zero: 9.993814255265587, alch_zero - forward valence: 9.993813524106281


In [24]:
print(f"nonalchemical_one: {nonalch_one_rp}, alch_one - reverse valence: {alch_one_rp - reverse_subtracted_valence_energy}")

nonalchemical_one: 6.0923632380951, alch_one - reverse valence: 6.092365753822985


In [25]:
old_positions.value_in_unit(unit.nanometers).astype(np.float64)

array([[ 1.88514099e-01, -1.03630655e-01, -1.12440391e-02],
       [ 8.06556195e-02, -4.63010632e-02,  5.61006330e-02],
       [ 2.92100519e-01, -1.63086548e-01,  6.08143508e-02],
       [ 7.63829425e-02, -4.84138317e-02,  1.95505470e-01],
       [ 2.87828594e-01, -1.65199250e-01,  2.00218752e-01],
       [ 1.79970011e-01, -1.07863985e-01,  2.67567605e-01],
       [ 1.91742301e-01, -1.02026664e-01, -1.19778946e-01],
       [ 3.79784105e-05,  3.04330788e-05, -3.29999239e-05],
       [ 3.76132488e-01, -2.07688287e-01,  8.41035042e-03],
       [-7.60346139e-03, -3.76650877e-03,  2.47908279e-01],
       [ 3.68484735e-01, -2.11495161e-01,  2.56312102e-01],
       [ 1.76652387e-01, -1.09545410e-01,  3.76138300e-01]])

## Last test is to conduct a `SOLVATED` transformation for some small molecules and test that the $\Delta U$ between the alchemical and nonalchemical systems is just the valence terms added and subtracted.

In [26]:
from perses.tests.utils import generate_solvated_hybrid_test_topology
from perses.rjmc.geometry import FFAllAngleGeometryEngine

In [27]:
#first, try it as a vacuum...so we treat environment atoms with the same nonbonded method as we did before...

In [28]:
#i suspect there may be a discrepancy in the way the hybrid topology factory sets the nonbonded method

In [29]:
top_proposal, old_positions= generate_solvated_hybrid_test_topology('benzene', 'toluene', propose_geometry = False)

DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:proposal_engine:Building new Topology object...
DEBUG:openmoltools.openeye:keep_confs was set to 1. Molecule positions will be reset.
DEBUG:openmoltools.openeye:MOL
DEBUG:parmed.openmm.parameters:Valid patch combinations:
INFO:proposal_engine:Proposed transformation would delete 1 atoms and create 4 atoms.


In [30]:
# top_proposal._old_system.getForce(3).setNonbondedMethod(0)
# top_proposal._new_system.getForce(3).setNonbondedMethod(0)

# top_proposal._old_system.removeForce(4)
# top_proposal._new_system.removeForce(4)

In [31]:
top_proposal._old_system.getForce(3).setUseDispersionCorrection(False)
top_proposal._new_system.getForce(3).setUseDispersionCorrection(False)

In [32]:
_geometry_engine = FFAllAngleGeometryEngine(metadata=None, use_sterics=False, n_bond_divisions=100, n_angle_divisions=180, n_torsion_divisions=360, verbose=True, storage=None, bond_softening_constant=1.0, angle_softening_constant=1.0, neglect_angles = False)

In [33]:
old_positions.value_in_unit(unit.nanometers)

array([[ 0.1885141 , -0.10363065, -0.01124404],
       [ 0.08065562, -0.04630106,  0.05610063],
       [ 0.29210052, -0.16308655,  0.06081435],
       ...,
       [ 0.08351727,  0.62023201,  0.09468868],
       [ 0.11641727,  0.66563201,  0.01708868],
       [ 0.15791727,  0.61833201,  0.15478868]])

In [34]:
_new_positions, _lp = _geometry_engine.propose(top_proposal, old_positions, beta)

In [35]:
_lp_rev = _geometry_engine.logp_reverse(top_proposal, _new_positions, old_positions, beta)

In [36]:
from perses.annihilation.new_relative import HybridTopologyFactory
HTF = HybridTopologyFactory(top_proposal, old_positions, _new_positions)

In [37]:
from perses.annihilation.lambda_protocol import RelativeAlchemicalState
import copy
from openmmtools.states import ThermodynamicState, CompoundThermodynamicState, SamplerState
from openmmtools import integrators

hybrid_system = HTF.hybrid_system
lambda_zero_alchemical_state = RelativeAlchemicalState.from_system(hybrid_system)
lambda_one_alchemical_state = copy.deepcopy(lambda_zero_alchemical_state)

lambda_zero_alchemical_state.set_alchemical_parameters(0.0)
lambda_one_alchemical_state.set_alchemical_parameters(1.0)

hybrid_system.getForce(7).setUseDispersionCorrection(False)

In [38]:
# Create thermodynamic states for the nonalchemical endpoints
thermodynamic_state = ThermodynamicState(hybrid_system, temperature = temperature)
nonalchemical_thermodynamic_states = {
    0: ThermodynamicState(top_proposal.old_system, temperature=temperature),
    1: ThermodynamicState(top_proposal.new_system, temperature=temperature)}

# Now create the compound states with different alchemical states
hybrid_thermodynamic_states = {0: CompoundThermodynamicState(thermodynamic_state,
                                                                   composable_states=[lambda_zero_alchemical_state]),
                                     1: CompoundThermodynamicState(copy.deepcopy(thermodynamic_state),
                                                                   composable_states=[lambda_one_alchemical_state])}

In [39]:
#create contexts
from openmmtools import integrators
from perses.tests.utils import compute_potential_components
nonalchemical_zero_context = nonalchemical_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_zero_context.setPositions(old_positions)
nonalchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_zero_context)]
print(f"nonalchemical_zero_energy components")
for force, energy in nonalchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_zero_energy: {sum(nonalchemical_zero_energy_components)}")
print()

nonalchemical_one_context = nonalchemical_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
nonalchemical_one_context.setPositions(_new_positions)
nonalchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(nonalchemical_one_context)]
print(f"nonalchemical_one_energy components")
for force, energy in nonalchemical_one_energy_components:
    print(f"\t{force}: {energy}")
#print(f"nonalchemical_one_energy: {sum(nonalchemical_one_energy_components)}")
print()
print()

alchemical_zero_context = hybrid_thermodynamic_states[0].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_zero_context.setPositions(HTF._hybrid_positions)
alchemical_zero_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_zero_context)]
print(f"alchemical_zero_energy components")
for force, energy in alchemical_zero_energy_components:
    print(f"\t{force}: {energy}")
#print(f"alchemical_zero_energy: {sum(alchemical_zero_energy_components)}")
print()

alchemical_one_context = hybrid_thermodynamic_states[1].create_context(integrators.LangevinIntegrator(temperature=temperature))
alchemical_one_context.setPositions(HTF._hybrid_positions)
alchemical_one_energy_components = [(force, energy*beta) for force, energy in compute_potential_components(alchemical_one_context)]
print(f"alchemical_one_energy components")
for force, energy in alchemical_one_energy_components:
    print(f"\t{force}: {energy}")

nonalchemical_zero_energy components
	HarmonicBondForce: 0.05787802063460886
	HarmonicAngleForce: 0.004459541747343244
	PeriodicTorsionForce: 0.00011703901332337944
	NonbondedForce: 593.0539160343947
	MonteCarloBarostat: 0.0

nonalchemical_one_energy components
	HarmonicBondForce: 1.0647467818966758
	HarmonicAngleForce: 2.9319137160494053
	PeriodicTorsionForce: 0.6844265480803335
	NonbondedForce: 594.6099277226332
	MonteCarloBarostat: 0.0


alchemical_zero_energy components
	CustomBondForce: 0.05780943147099217
	HarmonicBondForce: 1.0070059395892998
	CustomAngleForce: 0.003730209639256297
	HarmonicAngleForce: 2.9289128385182366
	CustomTorsionForce: 0.00011323521200709491
	PeriodicTorsionForce: 0.6843171166696428
	NonbondedForce: 602.7663547531444
	CustomNonbondedForce: -11.407151114482774
	CustomBondForce: 1.6947123957220158
	MonteCarloBarostat: 0.0

alchemical_one_energy components
	CustomBondForce: 0.05780943147099217
	HarmonicBondForce: 1.0070059395892998
	CustomAngleForce: 0.003730

In [40]:
forward_added_valence_energy = _geometry_engine.forward_final_context_reduced_potential - _geometry_engine.forward_atoms_with_positions_reduced_potential
reverse_subtracted_valence_energy = _geometry_engine.reverse_final_context_reduced_potential - _geometry_engine.reverse_atoms_with_positions_reduced_potential

In [41]:
nonalch_zero_rp = SamplerState.from_context(nonalchemical_zero_context).potential_energy*beta
nonalch_one_rp = SamplerState.from_context(nonalchemical_one_context).potential_energy*beta

alch_zero_rp = SamplerState.from_context(alchemical_zero_context).potential_energy*beta
alch_one_rp = SamplerState.from_context(alchemical_one_context).potential_energy*beta

In [42]:
print(f"nonalchemical_zero: {nonalch_zero_rp}, alch_zero - forward valence: {alch_zero_rp - forward_added_valence_energy}")

nonalchemical_zero: 593.0911796288584, alch_zero - forward valence: 593.093821410133


In [43]:
print(f"nonalchemical_one: {nonalch_one_rp}, alch_one - reverse valence: {alch_one_rp - reverse_subtracted_valence_energy}")

nonalchemical_one: 599.2687612433201, alch_one - reverse valence: 599.2689064288656


In [44]:
hybrid_system.getForce(7).getUseDispersionCorrection()

False

In [45]:
top_proposal._old_system.getForce(3).getUseDispersionCorrection()

False

In [46]:
top_proposal._new_system.getForce(3).getUseDispersionCorrection()

False